In [ ]:
%matplotlib inline

In [ ]:
! jt -t grade3 -nf opensans

<h1><center>1. Load Data</center></h1>



In [ ]:
import os
import json
import warnings
import pandas as pd

warnings.filterwarnings(action='ignore', message='numpy.dtype size changed')
warnings.filterwarnings(action='ignore', message='compiletime version 3.5 of module')

if not 'workbookDir' in globals():
    workbookDir = os.getcwd()
os.chdir(os.path.split(workbookDir)[0])

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

# Set the style of plots
plt.style.use('seaborn')
mpl.rcParams["axes.facecolor"] = 'w'
mpl.rcParams['grid.color'] = 'grey'
mpl.rcParams['grid.alpha'] = 0.1

mpl.rcParams['axes.linewidth'] = 0.3
mpl.rcParams['axes.edgecolor'] = 'grey'

mpl.rcParams['axes.spines.right'] = True
mpl.rcParams['axes.spines.top'] = True

In [ ]:
from synthesized.testing.plotting import plot_avg_distances
from synthesized.testing import UtilityTesting
from synthesized import HighDimSynthesizer

def run_baseline_evaluation(evaluation, evaluation_name, config, ax=None):
    config = configs["instances"][evaluation_name]
    evaluation.record_config(evaluation=evaluation_name, config=config)
    data = pd.read_csv(evaluation.configs[evaluation_name]['data'])
    data = data.drop(evaluation.configs[evaluation_name]['ignore_columns'], axis=1)
    
    display(Markdown("data length {}".format(len(data))))
    train, test = train_test_split(data, test_size=0.5)
    
    plot_avg_distances(train, test, evaluation, evaluation_name, ax=ax)
    
    try:
        synthesizer = HighDimSynthesizer(df=data)
        testing = UtilityTesting(synthesizer, test, train, test)
        utility = testing.utility(target=evaluation.configs[evaluation_name]['target'])
    except:
        utility = 1.0
    evaluation.record_metric(evaluation=evaluation_name, key='utility', value=utility)
    evaluation.record_metric(evaluation=evaluation_name, key='training_time', value=0.)



In [ ]:
from sklearn.model_selection import train_test_split
from synthesized.testing.evaluation import Evaluation
from synthesized.testing import plotting as syn_plot
from IPython.display import Markdown, display

evaluation_name = os.environ.get('evaluation_name', 'n/a')
branch = 'master'
revision = os.environ.get('evaluation_revision', 'n/a')
evaluation = Evaluation(branch=branch, revision=revision, group='baseline_evaluation', 
                        metrics_file= '../metrics_baseline.jsonl')

config_path = os.environ.get('baseline_evaluation_config_path','n/a')

with open(config_path, 'r') as f:
    configs = json.load(f)

for evaluation_name, config in configs['instances'].items():
    display(Markdown("## {}".format(evaluation_name)))
    run_baseline_evaluation(evaluation, evaluation_name, config)
    plt.show()


In [ ]:
evaluation.write_metrics(append=False)